### **LEVEL 1**

Если я закомментировал какой-то код, это скорее всего значит, что его выполнение занимает много времени. В таком случае более эффективным решением будет пропустить его и сразу применить сохраненные изменения, которые будут представлены в следующем блоке кода.

In [ ]:
!pip install trl
!pip install accelerate>=0.20.1
import torch
from torch.nn.functional import softmax
import pandas as pd
from trl import DPOTrainer
import random
import json
from transformers import Trainer, AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments, DataCollatorForLanguageModeling
import transformers
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DistilBertForSequenceClassification, DistilBertTokenizer
from datasets import Dataset
from scipy.stats import entropy
from collections import defaultdict
import numpy as np
import requests
from os import getcwd

In [ ]:
# !pip install wandb
# import wandb

# # чтобы отслеживать наше обучение воспользуемся библиотекой wandb
# # тут нужно будет войти в систему через логин и пароль
# !wandb login
# run = wandb.init(
#     project="T18",
# )

В середине этого блокнота будет раздел, в котором я создаю обучающую выборку с использованием SFT и файнтюниных моделек. Чтобы избежать ожидания в течение 3 часов, вместо этого я сохранил эти датасеты, и предлагаю вам загрузить их сразу здесь.

In [ ]:
url = 'https://raw.githubusercontent.com/AntonKorz/Aligment_IMDB/main/SFT_generated.csv'
directory = getcwd()
filename = directory + '/' + 'SFT_generated.csv'
r = requests.get(url)
f = open(filename, 'wb')
f.write(r.content)

url = 'https://raw.githubusercontent.com/AntonKorz/Aligment_IMDB/main/hinge_generated.csv'
directory = getcwd()
filename = directory + '/' + 'hinge_generated.csv'
r = requests.get(url)
f = open(filename, 'wb')
f.write(r.content)

url = 'https://raw.githubusercontent.com/AntonKorz/Aligment_IMDB/main/sigm_generated.csv'
directory = getcwd()
filename = directory + '/' + 'sigm_generated.csv'
r = requests.get(url)
f = open(filename, 'wb')
f.write(r.content)

Теперь подгрузим наши основные модельки.

gpt2_model_ref - моделька, которая будет использоваться в качестве референса к KL-loss регуляризации

In [ ]:
# Загрузка sft модели (GPT-2)
gpt2_model_name = "lvwerra/gpt2-imdb"
gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_model_name)
gpt2_model_ref = GPT2LMHeadModel.from_pretrained(gpt2_model_name)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model_name)
if gpt2_tokenizer.pad_token is None:
        gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Загрузка модели для оценки текстов (DistilBERT)
distilbert_model_name = "lvwerra/distilbert-imdb"
distilbert_model = DistilBertForSequenceClassification.from_pretrained(distilbert_model_name)
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(distilbert_model_name)

# Установка устройства (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt2_model.to(device)
gpt2_model_ref.to(device)
distilbert_model.to(device)

Теперь напишем функции, которые будут нам генерировать текст и оценивать его.

*Пришлось немного поэксперементировать с параметрами генерации, чтобы текст был осмысленным и при этом разным при одинаковом промте*

In [ ]:
# Функция для генерации текста
def generate_text(model, tokenizer, prompt, min_length=150, max_length=150, num_beams=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)

    # Генерация текста
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_beams=num_beams,
            no_repeat_ngram_size=2,
            do_sample=True,
            min_length=min_length,
            attention_mask=attention_mask,
            # pad_token_id=pad_token_id,
            temperature=4.0,
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Функция для оценки текста с использованием DistilBERT
def get_distilbert_logits(text):
    inputs = distilbert_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = distilbert_model(**inputs)
    logits = outputs.logits
    return -float(logits[0][0])

# Функция для оценки энтропии сгенерированного текста
def token_entropy(generations, tokenizer):
    stats = defaultdict(int)
    num_tokens = 0
    for example in generations:
        tokens = tokenizer.encode(example)
        for t in tokens:
            if t == tokenizer.pad_token_id:
                continue
            stats[t] += 1
            num_tokens += 1
        for k in stats.keys():
            stats[k] /= num_tokens
    return entropy(list(stats.values()))

Давайте сгенерируем 5 текстов, чтобы посмотреть, что все получается

In [ ]:
N = 5
for i in range(N):
    prompt = "Review:"
    generated_text = generate_text(gpt2_model, gpt2_tokenizer, prompt, min_length=50, max_length=50, num_beams=5)
    print(f"Generated Text {i+1}: {generated_text}")

    # Расчет reward с использованием DistilBERT
    logit = get_distilbert_logits(generated_text)
    print(f"DistilBERT Logit (Reward) for Text {i+1}: {logit}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: Review: This one's good but not worth renting. Not recommended to anyone unless you want a fun, interesting, and scary movie to see every once in a while and have no problem playing a poker game or watching a bad movie. In fact,
DistilBERT Logit (Reward) for Text 1: -1.3136324882507324



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 2: Review: A very intelligent and well written movie with very enjoyable acting including the director's superb work, excellent direction especially in this one as he puts you through it with enthusiasm and passion to the final part. If you like suspense film, suspense suspense thriller
DistilBERT Logit (Reward) for Text 2: 2.6438894271850586



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 3: Review: I liked the story and even the visuals. In addition, some great acting was included. If the film was released today, would you think it would deserve it to have some good special features like music, a theme park attraction? And if
DistilBERT Logit (Reward) for Text 3: 1.7295440435409546



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 4: Review: A true story not told by a master director. I have no idea how the story was achieved, how it ended up in, and why they changed things when there was an excuse to cut a good deal out of that film - all that
DistilBERT Logit (Reward) for Text 4: 0.4885181188583374

Generated Text 5: Review: 7/10, and that's it. I was the kind of writer who'd love to have that story tell you a little more in future films, though I'd be tempted to stop at such a level. If that didn't make
DistilBERT Logit (Reward) for Text 5: 0.7694262266159058



Теперь пришло время генерации нашей выборки, на которой мы будем обучать наши SLIC-HF и DPO. Для этого вначале зафиксируем небольшое количество промтов. Потом для каждого из них сгенерируем 50 ответов и оценим эти ответы. В итоге все красиво упакуем в dataframe и сохраним.

In [ ]:
top_prompts = [
    'This movie', 'Director', 'Actors', 'The', 'Cinema', 'Movie',
    'Plot summary', 'Cinematography', 'Genre', 'Script', 'Soundtrack', 'scene',
    'Character development', 'Cinematic techniques', 'Screenplay', 'Critical reception',
    'Box office', 'Movie quotes', 'Cinematic universe', 'Behind the scenes', 'Movie review',
    'Filmography', 'Casting', 'Suspense', 'Opening scene', 'Movie analysis',
    'Film production', 'Editing', 'Costume design', 'Special effects', 'Film score',
    'Film festivals', 'Film theory', 'Movie poster', 'Film adaptation', 'Screenwriting',
    'Film noir', 'Cinematic history', 'Documentary', 'Film technology', 'Art direction',
    'Foreign films', 'Film awards', 'Movie premiere', 'Film industry', 'Animated films'
]

In [ ]:
def get_generations(model, tokenizer, prompts, num_responses):
    data = []
    for prompt in prompts:
        for _ in range(num_responses):
            generated_text = generate_text(model, tokenizer, prompt, min_length=50, max_length=50, num_beams=5)
            logits = get_distilbert_logits(generated_text)
            data.append([prompt, generated_text, logits])

    df = pd.DataFrame(data, columns=["Prompt", "Generated Text", "Model Score"])
    return df

Сам код генерации выборки представлен ниже, однако он работает 3 часа, поэтому я предлагаю не запускать этот блок, а перейти сразу к следующему, где наша выборка просто автоматически достаётся.

In [ ]:
# # Параметры для генерации ответов
# num_responses = 50

# # Список для хранения данных
# data = []

# # Генерация ответов и оценка с помощью модели
# for prompt in top_prompts:
#     print(prompt)
#     for _ in range(num_responses):
#         generated_text = generate_text(prompt)
#         logits = get_distilbert_logits(generated_text)
#         data.append([prompt, generated_text, logits])

# # Создание датафрейма
# df = pd.DataFrame(data, columns=["Prompt", "Generated Text", "Model Score"])

# df.to_csv('SFT_generated.csv')

In [ ]:
train_df = pd.read_csv('SFT_generated.csv')

In [ ]:
train_df

,Unnamed: 0,Prompt,Generated Text,Model Score
0,0,This movie,"This movie is really bad, which is good to see...",-1.650084
1,1,This movie,"This movie was a joke, and should be avoided a...",-2.537610
2,2,This movie,"This movie, which had some of the best acting ...",2.021652
3,3,This movie,This movie has some great camera work and some...,2.474760
4,4,This movie,This movie was very good and I think that the ...,1.405530
...,...,...,...,...
2295,2295,Animated films,Animated films are not meant to be watched by ...,-0.957312
2296,2296,Animated films,"Animated films in the early 70's, I think this...",2.409756
2297,2297,Animated films,Animated films have always been somewhat of a ...,2.207893
2298,2298,Animated films,Animated films of the early 20th century could...,2.261572


Дальше разобъем нашу выборку на пары, как сказано в задании и статье и представим в нужном виде, как сказано в документации TRL.

In [ ]:
random.seed(42)

# Определяем функцию, которая преобразует DataFrame в словарь, содержащий парные данные
def df_to_dict(df, num_pairs):
    data = {"prompt": [], "chosen": [], "rejected": []}

    for prompt in top_prompts:
        new_df = df[df['Prompt'] == prompt]

        for _ in range(num_pairs):
            # Случайным образом выбираем два индекса и сортируем их для создания уникальной пар
            pair = tuple(sorted(random.sample(range(len(new_df)), 2)))
            data['prompt'].append(prompt)
            score1 = new_df.iloc[pair[0]]['Model Score']
            score2 = new_df.iloc[pair[1]]['Model Score']
            if score1 > score2:
              data['chosen'].append(new_df.iloc[pair[0]]['Generated Text'])
              data['rejected'].append(new_df.iloc[pair[1]]['Generated Text'])
            else:
              data['chosen'].append(new_df.iloc[pair[1]]['Generated Text'])
              data['rejected'].append(new_df.iloc[pair[0]]['Generated Text'])

    return data

train_dict = df_to_dict(train_df, 50)
train = Dataset.from_dict(train_dict)

In [ ]:
train

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2300
})

Дальше стоит определить наши параметры обучения. С ними мне пришлось немного поиграться. В основном пытался подобрать оптимальный коэффициент бета, чтобы после обучения текст получался не просто с высоким reward, но и осмысленным. Так если поставит его неудачно, то будет генерировать несвязанная последовательность из слов "прекрасный", "шедевр" и т. д.

Learning rate взял из статьи.

Также следует заметить, что внутри хинж лосса библиотеки TRL реализован нормализованный хинж лосс, а не хинж лосс представленный в статье SLIC-HF.

В ходе обучения основная проблема была в том, что loss падал, но не очень гладко. Это конечно проблема, которую я не понял, как решать. Но несмотря на это, модельки обучились хорошо.


Код обучения:

In [ ]:
# training_args = TrainingArguments(
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     learning_rate=1e-5,
#     remove_unused_columns=False,
#     output_dir="./test",
#     # report_to="wandb",
#     logging_steps=50,
# )


# dpo_trainer_hinge = DPOTrainer(
#     gpt2_model,
#     gpt2_model_ref,
#     beta=0.1,
#     train_dataset=train,
#     tokenizer=gpt2_tokenizer,
#     args=training_args,
#     loss_type ='hinge',
# )

# dpo_trainer_hinge.train()

Запушим полученную модель на huggingface

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()
# dpo_trainer_hinge.model.push_to_hub("AntonKorznikov/gpt2_hinge")

Скачаем запушенную модель с huggingface

In [ ]:
model_hinge = GPT2LMHeadModel.from_pretrained('AntonKorznikov/gpt2_hinge').to(device)

Посмотрим как работает моделька:

In [ ]:
# Генерация N текстов и расчет reward для каждого
N = 10
for i in range(N):
    prompt = "Review:"
    generated_text = generate_text(model_hinge, gpt2_tokenizer, prompt, min_length=50, max_length=50, num_beams=5)
    print(f"Generated Text {i+1}: {generated_text}")

    # Расчет reward с использованием DistilBERT
    logit = get_distilbert_logits(generated_text)
    print(f"DistilBERT Logit (Reward) for Text {i+1}: {logit}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: Review: The most beautiful I saw on DVD, I gave it a perfect thumbs up and had a tremendous collection of classic scenes for me! It's perfect as a companion to The Secret and a great fun watch for all ages, plus it has an
DistilBERT Logit (Reward) for Text 1: 2.6233034133911133



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 2: Review: I like this film! It plays very very well and it is beautiful. It is interesting both for an actor's ability to make the most of different kinds of people, as well as an artist in a way that we could relate to and
DistilBERT Logit (Reward) for Text 2: 2.6084604263305664



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generated Text 7: Review: 4 out 10: A good look at a young and beautiful girl who goes out, travels and is happy. And the way she feels with her mommy and grandmother, she's a real heartwarming story about the best of everyone. I
DistilBERT Logit (Reward) for Text 7: 2.5049376487731934



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 8: Review: The movie is so effective that the visuals are so original and it stays true to its look. There are moments moments where it has the magic of a film - like "Baggy Boots" - which may not necessarily be in all movies
DistilBERT Logit (Reward) for Text 8: 2.403818130493164



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 9: Review: I watched it on a Friday with some friends (it had great soundtrack and was really well played!). I loved it; the music is perfect in different ways. And it's just the kind you don't expect any different. Highly recommended and
DistilBERT Logit (Reward) for Text 9: 2.6237363815307617

Generated Text 10: Review: Excellent. A bit of an interesting film from a young talent, very good film for those with a small budget and are looking to be entertained by the good story that develops. Highly recommend! A must get. Recommended. I saw the film
DistilBERT Logit (Reward) for Text 10: 2.586859703063965



Теперь повторим все это только для sigmoid loss

In [ ]:
# dpo_trainer_sigm = DPOTrainer(
#     gpt2_model,
#     gpt2_model_ref,
#     beta=10,
#     train_dataset=train,
#     tokenizer=gpt2_tokenizer,
#     args=training_args,
#     loss_type ='sigmoid',
#     # peft_config=peft_config,
# )

# dpo_trainer_sigm.train()

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()
# dpo_trainer_sigm.model.push_to_hub("AntonKorznikov/gpt2_sigm")

In [ ]:
model_sigm = GPT2LMHeadModel.from_pretrained('AntonKorznikov/gpt2_sigm').to(device)

In [ ]:
# Генерация N текстов и расчет reward для каждого
N = 10
for i in range(N):
    prompt = "Review:"
    generated_text = generate_text(model_sigm, gpt2_tokenizer, prompt, min_length=50, max_length=50, num_beams=5)
    print(f"Generated Text {i+1}: {generated_text}")

    # Расчет reward с использованием DistilBERT
    logit = get_distilbert_logits(generated_text)
    print(f"DistilBERT Logit (Reward) for Text {i+1}: {logit}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: Review: I'd have liked to have gotten more from it's story. The acting and photography were superb.<br /><br />This movie was my first time looking at "the best of America, the people that I would never see again,
DistilBERT Logit (Reward) for Text 1: 2.2210001945495605



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 2: Review: "It is a very moving thriller. A big action thriller but is also entertaining from an inside angle of the world of film with all the beautiful action, great actors, good story, the best score." Watch it now on VHS/
DistilBERT Logit (Reward) for Text 2: 2.4629526138305664



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generated Text 7: Review: The first of two'specials', I watched an excellent first season so I was sure of its beauty even after the episode, as it did the first time, but with all of it a perfect adaptation. This has a strong bond of
DistilBERT Logit (Reward) for Text 7: 2.53186297416687



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 8: Review: It was fantastic, The movie is very realistic and full of great visuals. Even though it wasn't the most popular movie ever, there is always some good reviews from people who love a great film, because it is the kind of movie you
DistilBERT Logit (Reward) for Text 8: 2.5149428844451904



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 9: Review: 8 out of 10. Definitely see the movie as well as i have already heard the previews and has it rated high for the time period. It is so great and I think this film is very funny, yet it is an action, suspense
DistilBERT Logit (Reward) for Text 9: 2.5539870262145996

Generated Text 10: Review: Great work! The movie was quite entertaining! Thank you, Michael. Very well thought out, very well written & interesting as well! I highly recommend this one. This is definitely worth a 2-star viewing. Also, one of my
DistilBERT Logit (Reward) for Text 10: 2.614173173904419



Теперь настало время сгенерировать для каждой модельки небольшой нобор текстов, оценить их энтропию и положительную\негативную тональность.

In [ ]:
df_hinge = get_generations(model_hinge, tokenizer=gpt2_tokenizer, prompts=top_prompts, num_responses=3)
df_sigm = get_generations(model_sigm, tokenizer=gpt2_tokenizer, prompts=top_prompts, num_responses=3)

In [ ]:
print('SFT')
print('Энтропия: ', token_entropy(train_df["Generated Text"], gpt2_tokenizer))
print('Reward: ', np.mean([get_distilbert_logits(text) for text in train_df["Generated Text"]]))

print('\nHinge')
print('Энтропия: ', token_entropy(df_hinge["Generated Text"], gpt2_tokenizer))
print('Reward: ', np.mean([get_distilbert_logits(text) for text in df_hinge["Generated Text"]]))

print('\nSigmoid')
print('Энтропия: ', token_entropy(df_sigm["Generated Text"], gpt2_tokenizer))
print('Reward: ', np.mean([get_distilbert_logits(text) for text in df_sigm["Generated Text"]]))

SFT
Энтропия:  4.484833419615554
Reward:  0.5356063388667384

Hinge
Энтропия:  3.597510553890052
Reward:  2.4167400153650753

Sigmoid
Энтропия:  3.64248570190943
Reward:  2.4014521733574243


Какие можно сделать выводы на основании результатов, представленных выше? Да особо никаких  ☹
    
У нас есть две метрики по которым мы можем замерять качество нашего обучения:
1) Насколько хорошим, логичными, "человечным" генерируется текст. Самой простой метрикой этого является энтропия, которую мы и используем
2) Насколько положительные генерируются отзывы

И очевидно, что всегда придется искать некий trade-off. То есть чисто теоретически, мы могли бы сравнивать в таком сэттинге два различных лосса, если бы для каждого лосса сгенерировали большое количество моделей (меняя параметр бетта), множество которых образовывало на графике энтропия/качество некую кривую. И вот если бы кривая для hinge была строго выше кривой sigmoid тогда можно было бы говорить, что при равной энтропии у hinge выше оценки, или при равных оценках у hinge лучше получается тексты.

## **LEVEL 2**

Можно было поменять код из готовой библиотеки форконув их репозиторий с гитхаб, поменять соответствующий файл и сделать git+htps://github.com/....Но это у меня не получилось, вылетала ошибка, которую не мог исправить

Поэтому я решил просто сделать оболочку сферх класса, который мне нужно поменять.



Вот собственно код:

In [ ]:
import torch.nn.functional as F
from typing import Tuple


class MyDPOTrainer(DPOTrainer):

    def __init__(self, alpha: float = None, **kwargs):
        super().__init__(**kwargs)
        if alpha is None:
            self.alpha = 1
        else:
            self.alpha = alpha

    def dpo_loss(
        self,
        policy_chosen_logps: torch.FloatTensor,
        policy_rejected_logps: torch.FloatTensor,
        reference_chosen_logps: torch.FloatTensor,
        reference_rejected_logps: torch.FloatTensor,
        reference_free: bool = False,
    ) -> Tuple[torch.FloatTensor, torch.FloatTensor, torch.FloatTensor]:
        """Compute the DPO loss for a batch of policy and reference model log probabilities.

        Args:
            policy_chosen_logps: Log probabilities of the policy model for the chosen responses. Shape: (batch_size,)
            policy_rejected_logps: Log probabilities of the policy model for the rejected responses. Shape: (batch_size,)
            reference_chosen_logps: Log probabilities of the reference model for the chosen responses. Shape: (batch_size,)
            reference_rejected_logps: Log probabilities of the reference model for the rejected responses. Shape: (batch_size,)
            reference_free: If True, we ignore the _provided_ reference model and implicitly use a reference model that assigns equal probability to all responses.

        Returns:
            A tuple of three tensors: (losses, chosen_rewards, rejected_rewards).
            The losses tensor contains the DPO loss for each example in the batch.
            The chosen_rewards and rejected_rewards tensors contain the rewards for the chosen and rejected responses, respectively.
        """
        pi_logratios = policy_chosen_logps - policy_rejected_logps
        if reference_free:
            ref_logratios = 0
        else:
            ref_logratios = reference_chosen_logps - reference_rejected_logps

        logits = pi_logratios - ref_logratios

        # The beta is a temperature parameter for the DPO loss, typically something in the range of 0.1 to 0.5.
        # We ignore the reference model as beta -> 0. The label_smoothing parameter encodes our uncertainty about the labels and
        # calculates a conservative DPO loss.
        if self.loss_type == "sigmoid":
            losses = (
                -F.logsigmoid(self.beta * logits) * (1 - self.label_smoothing)
                - F.logsigmoid(-self.beta * logits) * self.label_smoothing
            )
        elif self.loss_type == "hinge":
            losses = torch.relu(1 - self.beta * logits)
        elif self.loss_type == "ipo":
            # eqn (17) of the paper where beta is the regularization parameter for the IPO loss, denoted by tau in the paper.
            losses = (logits - 1 / (2 * self.beta)) ** 2
        elif self.loss_type == "alpha":
            u1 = torch.exp(policy_chosen_logps - reference_chosen_logps)
            u2 = torch.exp(policy_rejected_logps - reference_rejected_logps)
            f = lambda x: (1 - x ** (-self.alpha)) / self.alpha
            losses = -F.logsigmoid(self.beta*f(u1) - self.beta*f(u2))
        elif self.loss_type == "JSD":
            u1 = torch.exp(policy_chosen_logps - reference_chosen_logps)
            u2 = torch.exp(policy_rejected_logps - reference_rejected_logps)
            f = lambda x: torch.log((2 * x) / (1 + x))
            losses = -F.logsigmoid(self.beta*f(u1) - self.beta*f(u2))
        elif self.loss_type == "FKL":
            u1 = torch.exp(policy_chosen_logps - reference_chosen_logps)
            u2 = torch.exp(policy_chosen_logps - reference_chosen_logps)
            f = lambda x: -1/x
            losses = -F.logsigmoid(self.beta*f(u1) - self.beta*f(u2))
        elif self.loss_type == "RKL":
            u1 = torch.exp(policy_chosen_logps - reference_chosen_logps)
            u2 = torch.exp(policy_chosen_logps - reference_chosen_logps)
            f = lambda x: torch.log(x) + 1
            losses = -F.logsigmoid(self.beta*f(u1) - self.beta*f(u2))
        else:
            raise ValueError(f"Unknown loss type: {self.loss_type}. Should be one of ['sigmoid', 'hinge', 'ipo', 'alpha', 'JSD']")

        chosen_rewards = self.beta * (policy_chosen_logps - reference_chosen_logps).detach()
        rejected_rewards = self.beta * (policy_rejected_logps - reference_rejected_logps).detach()

        return losses, chosen_rewards, rejected_rewards

Однако дальше я понял, что не успею сделать эксперименты с этими лосами, поэтому level 2 на этом заканчивается ☹

## **LEVEL 3**

Самые гениальные (глупенькие) идеи на свете или размышления вслух:

Интересно рассмотреть AI-aligment с точки зрения теории социального выбора. Это такой раздел экономики/математики, где исследуется, как множество различных агентов с различными предпочтениями на определенном множестве объектов должны выбрать один из этих объектов (выбрать 3 лучших\отранжировать), который будет удовлетворять большинство. В alignment, эта наука, возможно, может возникнуть в двух случаях:

1) Мы хотим сделать крутой ИИ с человеческими ценностями. Но какие ценности выбрать, если у разных людей ценности разные? То есть, как нам следует выбрать набор ценностей "победителей". Но это больше философский вопрос, по которому статейку вряд ли напишешь.

2) С более практической точки зрения, эта наука может возникнуть, если мы, например, решаем обучать несколько разных reward моделей для оценки различных характеристик (к примеру, отдельная reward модель для оценки агрессии ответа и модель для оценки логичной связанности ответа). Тогда у нас возможно будет возникать ситуация, когда одна модель имеет одни предпочтения на множестве ответов на промт, а другая модель - другие предпочтения. Как тогда нам нужно агрегировать их предпочтения? (Если это вообще нужно будет и проблемы такой на практике нет ¯\_(ツ)_/¯ ).


Интересно рассмотреть получше регуляризационный член в формуле лосса. Почему бы не брать квадрат от KL-дивергенции или любую другую монотонно возрастающую гладкую функцию на множестве положительных вещественных чисел, как это делают в классическом ML? Если же мы остаемся в контексте RL, то можно выбирать и не гладкие регуляризационные функции.

Так или иначе, это нас приводит к задаче определения эффективной относительно нашей задачи метрики на множестве нейросетей с одинаковой архитектурой. Можно начинать с тупого квадрата разности весов нейросетки. Можно смотреть, насколько отличаются латентные представления нейросеток. И можно, наверное, придумать еще миллион других метрик; задача в том, чтобы они хорошо отражали конкретно нашу задачу.

Ну тут уже очень практическая штука будет. Допустим, мы хотим научить LLM смешно шутить. Тогда нам не нужно будет собирать огромный датасет шуток и оценивать их. Это не круто. Круто сказать спасибо социальным сетям за то, что у них есть функция лайка. Другими словами, интересно понять, насколько может быть релевантным для обучения различных моделей (не только LLM) фидбек, который мы можем достать из социальных сетей.